# Qual o melhor meio de representar esso texto corrido para análise de dados?

## Importações

In [5]:
import pandas as pd
import numpy as np

import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

pd.set_option('display.max_colwidth', None)

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Leitura do arquivo salvo na primeira parte do processo

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/leoferro/analise_textual_capivari/main/data/raw_data.csv')

In [3]:
df.head()

,Protocolo,Data,Assunto,Situação,Regime,Autoria
0,00594/2024,20/02/2024,Solicita informações sobre nebulização de inseticida no município. Entre outras informações.,Protocolada,Ordinário,MARCOS SABINO FERNANDES
1,00593/2024,20/02/2024,Moção de Repúdio ao Excelentíssimo Presidente da República Luiz Inácio Lula da Silva pelas declarações sobre Israel.,Protocolada,Ordinário,JOSÉ EDUARDO DE CAMPOS PACHECO
2,00580/2024,19/02/2024,Solicita melhorias nas ruas do bairro Cancian,Protocolada,Ordinário,THIAGO JUNIOR ANESIO BRAGGION
3,00533/2024,19/02/2024,Solicita troca de lâmpadas queimadas na rodoviária,Protocolada,Ordinário,DENILTON ROCHA DOS SANTOS
4,00526/2024,19/02/2024,Solicita melhorias na ponte (Pontinha do SAAE) entre a rotatória do Ginásio Ronaldão e o bairro Estação,Protocolada,Ordinário,RUAN HENRIQUE PELLEGRINI BRAGGION


# Transformações de texto para vetor

Serão testadas algumas abordagens diferentes para a análise e seus significados

## Frequencia do termo

Essa é a transformação base para as proximas. Medimos quantas vezes a palavra aparece dentro de cada objeto, no caso cada assunto, e retorna o esse número em uma matriz onde as linhas são cada cada linha do dataframe original, o objeto analisado e as palavras vão para colunas. Alguns parametros são utilizados para evitar algumas palavras desnecessárias com pouco significado.

In [19]:
cv = CountVectorizer(
    strip_accents='ascii',                      # Desconsiderar acentos
    lowercase=True,                             # Ignorar caixa alta
    stop_words = stopwords.words('portuguese'), # Retirar palavras que não carregam significado
    ngram_range=(1,1),                          # Considerar apenas unidades de palavras
    min_df=5                                    # Retornar apenas palavras que aparecem pelo menos 5 vezes
).fit(df.Assunto)

In [20]:
out = pd.DataFrame(cv.transform(df.Assunto).toarray(), columns = cv.get_feature_names_out())
out.head()

,00,000,001,003,004,005,007,008,009,010,...,vila,vinicius,visando,vista,vistoria,vitorio,xii,xv,zona,zoonoses
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Lista das 50 palavras com maior frequencia.

A partir desse tipo de transformação temos as palavras com maior frequencia muito genéricas e que muitas vezes não trazem nenhum significado para os textos em sí.

In [21]:
" - ".join(pd.DataFrame(out.sum().sort_values(ascending=False)).head(50).index.to_list())

'2023 - bairro - solicita - rua - indicacao - sobre - informacoes - vereador - excelentissimo - prefeito - requerimento - municipal - solicitando - lei - avenida - sao - projeto - limpeza - capivari - jardim - 2022 - jose - instalacao - autoria - mocao - apoio - manutencao - municipio - demais - pares - estudos - adicional - joao - melhorias - santa - antonio - credito - poder - construcao - silva - buraco - braggion - tapa - especial - doutor - implantacao - escola - eduardo - vereadora - secretaria'

# TF com frequencia relativa

Podemos a partir do nosso exemplo acima criar uma matriz de frequencias relativas, em relação à cada objeto, ou seja se uma palavra aparecer mais de uma vez dentro de um objeto ou se o objeto tiver poucas palavras essa palavra receberá um valor maior. Essa possibilidade nos trás a possibilidade superficial de compreender a importância da palavra dentro da frase.

In [50]:
# Registrar o total por linha em uma variavel
totais = out.sum(axis=1)
totais.loc[totais == 0] = 1

# Transformando em um vetor para poder operar sobre cada coluna
totais =  totais.to_numpy().reshape(-1,1)

# Dividir cada linha pela quantidade total de palavras para ter a frequencia relativa
out_freq = (out/totais)

#Exemplo com a palavra zona
out_freq[out_freq['zona']>0]

,00,000,001,003,004,005,007,008,009,010,...,vila,vinicius,visando,vista,vistoria,vitorio,xii,xv,zona,zoonoses
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.090909,0.0
410,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.062500,0.0
707,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.058824,0.0
905,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.090909,0.0
906,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0
923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.062500,0.0


# DF - Frequencia no documento

Essa medida trata principalmente de entender qual a frequencia que dada palavra acontece dentro dos objetos apresentados dna base de dados. Ela parte do pressiposto de não levar em conta se a palavra aparece uma ou mais, é apenas se aparece ou não. Serve principalmente para compreender se dada palavra é muito comum ou não dentro do conjunto de dados.

É possivel perceber que apesar de algumas diferenças os valores se assemelham ao agrupamento da primeira vetorização. A diferença é apenas que aqui eles podem ser contados apenas uma vez.

In [61]:
# Transformação de valores únicos para representar booleanamente se aparece ou não no documento
out_df = (out>0).astype(int)

# Contagem de quantas vezes cada coluna aparece
out_df = out_df.sum()

# Print dos 50 primeiros valores
" - ".join(list(out_df.sort_values(ascending  =False).head(50).index))

'2023 - bairro - solicita - indicacao - rua - sobre - informacoes - vereador - excelentissimo - prefeito - requerimento - solicitando - municipal - avenida - sao - lei - projeto - capivari - limpeza - jardim - instalacao - jose - autoria - mocao - 2022 - manutencao - municipio - demais - apoio - pares - estudos - adicional - joao - melhorias - santa - antonio - credito - poder - construcao - silva - buraco - tapa - braggion - especial - implantacao - doutor - escola - eduardo - vereadora - localizado'

# TF-IDF

Essa métrica utiliza a frrequencia do termo dentro os documentos juntamente com a proporção inversa da quantidade de vezes em que o termo ocorre em outros documentos fazendo com que termos muito genéricos sofram uma penalidade.

Essa metrica representa a TF (frequencia em porcentagem em que a palavra ocorre no documento) poderada pela IDF (frequencia inversa do numero de documentos que contem essa palavra)

A utilização dela se torna importante principalmetne por que podemos ponderar a frequencia relativa do termo utilizada acima pela inversa da frequencia d documento, fazendo com que palavras mais importantes no documento e não comuns no texto recebam valores maiores.




In [62]:
tfidf = TfidfVectorizer(
    stop_words = stopwords.words('portuguese'),
    min_df = .01,
    max_df = .1
    )
vec   = tfidf.fit_transform(df.Assunto)

out_vec = pd.DataFrame(vec.toarray(), columns = tfidf.get_feature_names_out())
out_vec.head()

,00,2021,2022,abertura,abrir,academia,acesso,adicional,alegre,altera,...,viabilize,vila,vinícius,visando,xii,água,área,árvores,ênio,ônibus
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Lista das 50 palavras com maior frequencia.

Ainda possui palavras com pouco peso mas já temos algumas mudanças por conta da ponderaçã com o idf.

In [ ]:
" - ".join(pd.DataFrame(out_vec.agg(['mean']).T.sort_values('mean',ascending=False)).head(50).index.to_list())

'lei - avenida - projeto - limpeza - capivari - manutenção - município - instalação - jardim - 2022 - josé - estudos - antônio - melhorias - santa - adicional - buraco - joão - moção - implantação - tapa - escola - construção - apoio - autoria - demais - silva - centro - secretaria - pares - doutor - poder - vereadora - especial - eduardo - localizado - servidores - braggion - deficiência - escolas - número - vila - área - executivo - ruas - saúde - santo - água - batista - ônibus'

### Análise dos textos que possuem a palavra academia.

Vemos que o que teve o maior TF-IDF teve uma questão explicita em relação à academia direta. Então por ter menos palavras dentro do documento e possuir a palavra academia acabou recebendo um tf-idf maior

In [63]:
pd.concat([
    df.loc[out_vec['academia']>0, ['Assunto']],
    out_vec.loc[out_vec['academia']>0, ['academia', 'calçada']]
    ], axis =1).sort_values('academia', ascending = False)

,Assunto,academia,calçada
144,"Solicita estudos sobre a viabilidade de oferecer uma academia gratuita, para a população.",0.816791,0.000000
1070,"INDICAÇÃO N° 73 - MELHORIAS NA CALÇADA ONDE FORAM INSTALADOS OS BRINQUEDOS E A ACADEMIA ADAPTADAS, NO BAIRRO VILA IZILDINHA.",0.495440,0.476338
946,INDICAÇÃO Nº 160 - ACADEMIA AO AR LIVRE NA ÁREA DE LAZER PÂMELA BORGES BORTOLASO.,0.471618,0.000000
1197,"Nº 005 - ÁREA DE LAZER COM PLAYGROUND, ACADEMIA AO AR LIVRE NO BAIRRO SANTA TERESA D'ÁVILA.",0.445983,0.000000
850,"INDICAÇÃO 215/2023 - PARA IMPLANTAR UMA ACADEMIA AO AR LIVRE, NO BAIRRO SÃO JOÃO BATISTA.",0.437155,0.000000
625,"INDICAÇÃO 349/2023 - - FECHAR O TUBO DE SUPORTE DOS EQUIPAMENTOS DE GINASTICA DA ACADEMIA AO AR LIVRE LOCALIZADA NA PRAÇA DA BÍBLIA, PARA NÃO DEIXAR ÁGUA PARADA OU ENFERRUJAR O EQUIPAMENTO.",0.431009,0.000000
237,"Solicita implantação de um parquinho, academia adaptada, bancos, bebedouros no Bairro Vila Fátima, na esquina da Rua Romário de Andrade com a Avenida Doutor José de Carvalho Junior.",0.421101,0.000000
525,Restauração da academia ao ar livre existente na Área de Lazer Gilson Gozzo e a instalação de lixeiras.,0.417198,0.000000
776,"REQUERIMENTO N° 109/2023, DO VEREADOR ELISSANDRO DA SILVA LEITE (SANDRO MODAS), AO EXCELENTÍSSIMO PREFEITO, SOLICITANDO INFORMAÇÕES SOBRE PERIOCIDADE NA MANUTENÇÃO DE EQUIPAMENTOS DE ACADEMIA AO AR LIVRE.",0.412141,0.000000
645,"REQUERIMENTO N° 143/2023, DO VEREADOR VINÍCIUS SCARSO, AO EXCELENTÍSSIMO PREFEITO, SOLICITANDO INFORMAÇÕES SOBRE IMPLANTAÇÃO DE ACADEMIA DE GINÁSTICA AO AR LIVRE, NO BAIRRO JARDIM SÃO PEDRO.",0.396829,0.000000


In [ ]:
out.to_csv    ('cont_vec.csv')
out_vec.to_csv('tf_idf.csv')